# Data Analysis - Irradiation of a Fibre Optic Sensor
Measuring humidity in radioactive environments is challenging because - as you might expect - the radiation is massively damaging to sensors. Most commercial humidity sensors are electronic, get wiped out by even a small dose of radiation. 
<!-- This presents a problem for measuring humidity in many environments with high radiation - in space, in nuclear industries and in high energy physics experiments. -->
<!-- (What commercial sensors?
How much radiation kills them??) -->

However, recently a new type of technology has emerged for these cases - fibre optic sensors (FOS). These can survive radiation doses up to the Mega-Gray (MGy) level while still giving a clear signal. Even better, the change in the signal itself can actually be used to measure the radiation dose. Combined with recent breakthroughs in FOS for measuring humidity, these sensors present an exciting potential product with applications in areas such as nuclear industry, high energy physics research, and space travel.
<!-- Link to FOS radiation studies
Link to LPG humidity studies
How much damage would MGy do to a person? -->

This project will take you through analysis of a sensor under irradiation, using real data recorded from the IRRAD facility in CERN, Geneva, using bleeding-edge long period grating sensors.
<!-- Link IRRAD -->